# Memgraph QA

This notebook shows how to use LLMS to provide a natural language interface to a Memgraph graph database that you can query with the Cypher query language.

You will need to have a running instance of [Memgraph](https://memgraph.com). The easiest option is to create an instance using a Docker container using the following script:

docker run 
    -it -p 7687:7687 
    -p 7444:7444 
    -p 3000:3000 
    -e MEMGRAPH="--bolt-server-name-for-init=Neo4j/" 
    memgraph/memgraph-platform

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import MemgraphGraph

In [38]:
graph = MemgraphGraph(
    url="bolt://127.0.0.1:7687", username="username", password="password"
)

## Seeding the database

Using the Datasets section of the menu in [Memgraph Lab](https://memgraph.com/lab), you can choose Load Dataset on the Game of Thrones deaths dataset to easily create a sample database to query. The following code assumes you have loaded this dataset.

## Refresh graph schema information

If the schema of the database changes, you can refresh the schema information need to generate Cypher statements

In [39]:
graph.refresh_schema()
print(graph.get_schema)

Node properties are the following:
Node name: 'Character', Node properties: [{'property': 'name', 'type': 'str'}]
Node name: 'Location', Node properties: [{'property': 'name', 'type': 'str'}]
Node name: 'Allegiance', Node properties: [{'property': 'name', 'type': 'str'}]
Node name: 'Episode', Node properties: [{'property': 'number', 'type': 'int'}, {'property': 'name', 'type': 'str'}, {'property': 'imdb_rating', 'type': 'float'}]
Node name: 'Season', Node properties: [{'property': 'number', 'type': 'int'}]
Node name: 'Death', Node properties: [{'property': 'order', 'type': 'int'}]

Relationship properties are the following:
Relationship Name: 'KILLED', Relationship Properties: [{'property': 'method', 'type': 'str'}, {'property': 'count', 'type': 'int'}]

The relationships are the following:
['(:Character)-[:LOYAL_TO]->(:Allegiance)']
['(:Character)-[:VICTIM_IN]->(:Death)']
['(:Character)-[:KILLER_IN]->(:Death)']
['(:Character)-[:KILLED]->(:Character)']
['(:Episode)-[:PART_OF]->(:Season

## Querying the graph

We can now use GraphCypherQAChain to ask questions of the graph.

In [40]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0, model="gpt-4"),
    graph=graph,
    verbose=True
)

In [41]:
results = chain.run("Where did Jon Snow die?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Character {name: 'Jon Snow'})-[:VICTIM_IN]->(d:Death)-[:HAPPENED_IN]->(l:Location)
RETURN l.name
Full Context:
[{'l.name': 'Castle Black'}]

> Finished chain.


In [42]:
print(results)

Jon Snow died at Castle Black.


## Limit the number of results

You can limit the number of results from the GraphCypherQAChain by using the top_k parameter. The default is 10. 

In [43]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), 
    graph=graph, 
    verbose=True,
    top_k=1
)

chain.run("Which characters had the most allegiances?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Character)-[:LOYAL_TO]->(a:Allegiance)
RETURN c.name, COUNT(a) AS allegianceCount
ORDER BY allegianceCount DESC
LIMIT 1
Full Context:
[{'c.name': 'Obara Sand', 'allegianceCount': 2}]

> Finished chain.


'Obara Sand is one of the characters who had the most allegiances, with a count of 2.'

## Return intermediate results

You can return intermediate steps from GraphCypherQAChain using the `return_intermediate_steps` parameter.

In [44]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), 
    graph=graph, 
    verbose=True,
    return_intermediate_steps=True
)

results = chain("Show me the characters with allegiance to the white walkers.")
print(f"Intermediate steps: {results['intermediate_steps']}")
print(f"Final answer: {results['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Character)-[:LOYAL_TO]->(a:Allegiance)
WHERE a.name = 'White Walkers'
RETURN c.name
Full Context:
[{'c.name': 'White Walker'}, {'c.name': 'Viserion'}, {'c.name': 'Wight'}, {'c.name': 'Undead Polar Bear'}, {'c.name': 'Night King'}, {'c.name': 'Giant Wight'}]

> Finished chain.
Intermediate steps: [{'query': "MATCH (c:Character)-[:LOYAL_TO]->(a:Allegiance)\nWHERE a.name = 'White Walkers'\nRETURN c.name"}, {'context': [{'c.name': 'White Walker'}, {'c.name': 'Viserion'}, {'c.name': 'Wight'}, {'c.name': 'Undead Polar Bear'}, {'c.name': 'Night King'}, {'c.name': 'Giant Wight'}]}]
Final answer: The characters with allegiance to the White Walkers are White Walker, Viserion, Wight, Undead Polar Bear, Night King, and Giant Wight.


## Return direct results

You can return direct results from GraphCypherQAChain using the `return_direct` parameter.

In [45]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), 
    graph=graph, 
    verbose=True,
    return_direct=True
)

chain.run("Who killed Ned Stark?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (killer:Character)-[:KILLED]->(victim:Character {name: 'Ned Stark'})
RETURN killer.name

> Finished chain.


[{'killer.name': 'Ilyn Payne'}]